import libraries

In [148]:
import pandas as pd
import json
from collections import defaultdict
from bs4 import BeautifulSoup
import requests
import os
import re
from collections import Counter
from functools import reduce
from tqdm.notebook import tqdm

import custom functions

In [174]:
# Specify the full path to the module (including the filename)
module_path = '/content/drive/MyDrive/HM3-ADM/HW3_ADM/parser.py'

# Import the module using importlib (a workaround if the standard import doesn't work)
import importlib.util
spec = importlib.util.spec_from_file_location("parser", module_path)
parser = importlib.util.module_from_spec(spec)
spec.loader.exec_module(parser)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#[1.3] Parse downloaded pages



At this point, you should have all the HTML documents about the master's degree of interest, and you can start to extract specific information. The list of the information we desire for each course and their format as desiried.

#Access the stored HTML pages

In [ ]:
%pip install google-colab-shell
# import the module once
from google_colab_shell import getshell

  Preparing metadata (setup.py) ... done
  Created wheel for google-colab-shell: filename=google_colab_shell-0.2-py3-none-any.whl size=4107 sha256=80e086c86d11d45d2f29c9a491589816fae1262d0347b13dc5ea33ecab08cf12
  Stored in directory: /root/.cache/pip/wheels/f9/36/65/95dd4599be065418a9fe1f482674c8e716ce540f3f484681d2
Successfully built google-colab-shell


used command

---mettere modalità python--
git clone https://github.com/marinazanoni/nome_provvisorio.git

after accesing the repository

In [183]:
def parser(html_page):

    # Define your default values here
    default_values = {
    'courseName': None,
    'universityName': None,
    'facultyName': None,
    'isItFullTime': None,
    'description': None,
    'startDate': None,
    'fees': None,
    'modality': None,
    'duration': None,
    'city': None,
    'administration': None,
    'country': None,
    'url': None
}

    # Create a defaultdict with default values
    info = defaultdict(lambda: default_values)

   # Extract the text (HTML)
    with open(html_page, 'r', encoding='utf-8') as file:
        html_content = file.read()
    page_soup = BeautifulSoup(html_content, 'html.parser')

    # COURSE NAME
    page_links = page_soup.find_all('h1', {'class':'text-white course-header__course-title'})
    if page_links:
        first_h1_text = page_links[0].text
        info['courseName'] = str(first_h1_text)
    #else:
        # Handle the case when no 'h1' with 'data-permutive-title' is found
    #    print("No course name found.")
    #courseName = str(first_h1_text)

    # UNIVERSITY NAME
    # Extracting our tag of interest for the Name
    page_links = page_soup.find_all('a', {'class': 'course-header__institution'})
    # Extracting the name of the course as string and print it
    info['universityName'] = str(page_links[0].contents[0])

    # FACULTY NAME
    page_links = page_soup.find_all('a', {'class': 'course-header__department'})
    info['facultyName'] = str(page_links[0].contents[0])

    # FULL TIME
    page_links = page_soup.find_all('a', {'class': 'inheritFont'})
    info['isItFullTime'] = str(page_links[0].contents[0])

    # SHORT DESCRIPTION
    page_links = page_soup.find('div', {'id': 'Snippet'})
    info['description'] = str(page_links.get_text(separator='\n').strip())

    # STARTING DATE
    page_links = page_soup.find('span', {'class': 'key-info__start-date'})
    info['startDate'] = str(page_links.get_text())

    # FEES
    page_links = page_soup.find('a', {'class': 'noWrap inheritFont'})
    page_links = page_soup.find('div', {'class': 'course-sections__fees'})
    if page_links:
        fees_text = page_links.get_text(separator='\n').strip()
        # Remove "Fees" from the text content
        info['fees'] = fees_text.replace('Fees', '').strip()


    # MODALITY
    page_links = page_soup.find('span', {'class': 'key-info__content key-info__qualification py-2 pr-md-3 text-nowrap d-block d-md-inline-block'})
    # Get all elements within the span using find_all
    elementsWithinSpan = page_links.find_all('a')
    info['modality'] = ' '.join([element.text.strip() for element in elementsWithinSpan])

    # DURATION
    page_links = page_soup.find('span', {'class':'key-info__content key-info__duration py-2 pr-md-3 d-block d-md-inline-block'})
    info['duration']=str(page_links.text)

    # CITY
    page_links = page_soup.find('a', {'class':'card-badge text-wrap text-left badge badge-gray-200 p-2 m-1 font-weight-light course-data course-data__city'})
    info['city']=str(page_links.text)

    # ADMINISTRATION
    page_links = page_soup.find('span', {'class':'course-header__online-flag badge bg-white p-2 h6 shadow-sm mr-1'})
    if page_links == None:
        info['administration'] = "On Campus"
    else:
        info['administration']=str(page_links.text)

    # COUNTRY
    page_links = page_soup.find('a', {'class':'card-badge text-wrap text-left badge badge-gray-200 p-2 m-1 font-weight-light course-data course-data__country'})
    info['country']=page_links.text

    #URL
    page_links = page_soup.find('link')
    info['url'] = page_links.get('href')


    return(pd.DataFrame([info]))


*Searching for a tipical hidden file and delete it in order not to have it when exctracting the information from the folder*

In [ ]:
file_path_to_remove = '/content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/.DS_Store'
# Delete the hidden file
os.remove(file_path_to_remove)
print(f"The file {file_path_to_remove} has been deleted.")

*Applying the function to our html pages*

In [198]:
# Specifing the path of the folder
folder_path = '/content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages'

# List all files in the folder
files = os.listdir(folder_path)

parsed_dfs = []

# Iterate through all folders and subfolders using os.walk
for folder_path, _, file_names in os.walk(folder_path):
    # Check if there are files in the current folder
    if file_names:
        # Iterate through each file in the current folder
        for file_name in file_names:
            file_path = os.path.join(folder_path, file_name)

            # Store the information only is the dictionary is not empty (has at list a name course)
            try:
                # Parse the file and append the result to the list
                parsed_df = parser(file_path)
                parsed_dfs.append(parsed_df)
            except Exception as e:
                # Print the file path when an exception occurs
                print(f"Error parsing file: {file_path}")
                # print(f"Error details: {e}")

# Concatenate all DataFrames in the list
concatenated_df = pd.concat(parsed_dfs, ignore_index=True)


Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_118/course_1765.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_119/course_1772.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_128/course_1910.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_140/course_2086.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_196/course_2929.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_196/course_2931.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_215/course_3213.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_291/course_4357.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_293/course_4395.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_2

In [207]:
concatenated_df['url'][3]

'https://www.findamasters.com/masters-degrees/course/applied-economics-banking-and-financial-markets-online-msc/?i280d8352c56675'

In [209]:
concatenated_df['url'].nunique()

5979

*Results of the merged pages*

In [ ]:
# Display the concatenated DataFrame
concatenated_df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,administration,country,url
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,3D visualisation and animation play a role in ...,September,Please see the university website for further ...,MSc,1 year full-time,Glasgow,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
1,Air Quality Solutions - MSc,University of Leeds,Institute for Transport Studies,Full time,Up to 7 million people are estimated to die ev...,September,"UK: £12,500 (Total) \nInternational: £28,750 (...",MSc,"1 year full time, 2 or 3 years part-time",Leeds,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,Full time,The Analytical Toxicology MSc is a unique stud...,See Course,Please see the university website for further ...,MSc,Full-time: One year,London,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
3,Applied Computer Science and Artificial Inte...,University of Bradford,Faculty of Engineering & Digital Technologies,Full time,Computer science is the foundation of many exc...,"September, January",Please see the university website for further ...,MSc,1 Year Full Time / 2 Years Part Time,Bradford,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
4,Applied Economics (Banking and Financial Mar...,University of Bath,University of Bath Online,Part time,From political uncertainty to finance and recr...,"September, January",Cost per 10 credits £722* (10% alumni discount...,MSc,2 years and 6 months full time,Bath,Online,United Kingdom,https://www.findamasters.com/masters-degrees/c...


In [ ]:
concatenated_df.shape

(5979, 13)

In [ ]:
print(6000-5979, 'were not valid pages')

21 were not valid pages


In [ ]:
parsed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   courseName      1 non-null      object
 1   universityName  1 non-null      object
 2   facultyName     1 non-null      object
 3   isItFullTime    1 non-null      object
 4   description     1 non-null      object
 5   startDate       1 non-null      object
 6   fees            1 non-null      object
 7   modality        1 non-null      object
 8   duration        1 non-null      object
 9   city            1 non-null      object
 10  administration  1 non-null      object
 11  country         1 non-null      object
 12  url             1 non-null      object
dtypes: object(13)
memory usage: 232.0+ bytes


In [210]:
# Specify the path where you want to save the .tsv file
tsv_file_path = '/content/drive/MyDrive/HM3-ADM/HW3_ADM/MasterDegrees.tsv'

# Save the DataFrame to a .tsv file
concatenated_df.to_csv(tsv_file_path, sep='\t', index=False)

print(f".tsv file saved at: {tsv_file_path}")

.tsv file saved at: /content/drive/MyDrive/HM3-ADM/HW3_ADM/MasterDegrees.tsv


### *saving singularly the information about each master (html page)*

In [ ]:

# Iterate through each DataFrame in the list
for index, parsed_df in enumerate(parsed_dfs):
    # Iterate through each row in the DataFrame
    for row_index, row in parsed_df.iterrows():
        # Replace NaN values with empty strings
        single_row = row.fillna(' ')
        # Specify the path where you want to save the .tsv file for the current row
        tsv_file_path = f'/content/drive/MyDrive/HM3-ADM/HW3_ADM/Courses/course_{index}.tsv'
        # Save the single row DataFrame to a .tsv file
        with open(tsv_file_path, 'w') as file:
            file.write('\t'.join(single_row))
        # Uncomment to see all the tsv files
        #print(f".tsv file for row {index}, row {row_index} saved at: {tsv_file_path}")

#Preproccessing

In [103]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem import *

# Download the stopwords dataset if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#[2.0.0]

In [140]:
df = pd.read_csv('/content/drive/MyDrive/HM3-ADM/HW3_ADM/MasterDegree.tsv',sep='\t')
df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,administration,country,url
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,3D visualisation and animation play a role in ...,September,Please see the university website for further ...,MSc,1 year full-time,Glasgow,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
1,Air Quality Solutions - MSc,University of Leeds,Institute for Transport Studies,Full time,Up to 7 million people are estimated to die ev...,September,"UK: £12,500 (Total) \nInternational: £28,750 (...",MSc,"1 year full time, 2 or 3 years part-time",Leeds,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,Full time,The Analytical Toxicology MSc is a unique stud...,See Course,Please see the university website for further ...,MSc,Full-time: One year,London,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
3,Applied Computer Science and Artificial Inte...,University of Bradford,Faculty of Engineering & Digital Technologies,Full time,Computer science is the foundation of many exc...,"September, January",Please see the university website for further ...,MSc,1 Year Full Time / 2 Years Part Time,Bradford,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
4,Applied Economics (Banking and Financial Mar...,University of Bath,University of Bath Online,Part time,From political uncertainty to finance and recr...,"September, January",Cost per 10 credits £722* (10% alumni discount...,MSc,2 years and 6 months full time,Bath,Online,United Kingdom,https://www.findamasters.com/masters-degrees/c...


## Stemming

In [141]:
# Make a copy of the original DataFrame
processed_df = df.copy()

# stemmer
stemmer = PorterStemmer()

processed_df['description'] = processed_df.description.apply(lambda row: ' '.join([stemmer.stem(word) for word in row.split(' ')]))

##Lower, removing punctuation and stepwords

In [142]:
stop_words = set(stopwords.words('english'))

# Function to remove stopwords and punctuation from a text
def clean(text):
    """
    The following function returns the filtered element for each column of a dataframe.
    Filtering operation consists in removing punctuation and removing stopwords given text with lower case
    """
    words = word_tokenize(str(text))
    # Remove punctuation using NLTK and string.punctuation
    filtered_words = [word for word in words if word not in string.punctuation + "’"]
    # Remove stopwords
    filtered_words = [word for word in filtered_words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Selecting columns
text_columns = processed_df.select_dtypes(include='object').columns
# Lower
processed_df[text_columns] = processed_df[text_columns].apply(lambda x: x.str.lower() if x.dtype == 'O' else x)
# Applying clean function to all the valid columns
processed_df[text_columns] = processed_df[text_columns].applymap(clean)


In [53]:
processed_df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,administration,country,url
0,3d design virtual environments msc,glasgow caledonian university,school engineering built environment,full time,3d visualis anim play role mani areas popular ...,september,please see university website information fees...,msc,1 year full-time,glasgow,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...
1,air quality solutions msc,university leeds,institute transport studies,full time,7 million peopl estim die everi year due harm ...,september,"uk £12,500 total international £28,750 total",msc,1 year full time 2 3 years part-time,leeds,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...
2,analytical toxicology msc,king college london,faculty life sciences medicine,full time,analyt toxicolog msc uniqu studi cours integr ...,see course,please see university website information fees...,msc,full-time one year,london,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...
3,applied computer science artificial intelligen...,university bradford,faculty engineering digital technologies,full time,comput scienc foundat mani excit cutting-edg t...,september january,please see university website information fees...,msc,1 year full time 2 years part time,bradford,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...
4,applied economics banking financial markets on...,university bath,university bath online,part time,polit uncertainti financ recruit demands econo...,september january,cost per 10 credits £722 10 alumni discount pa...,msc,2 years 6 months full time,bath,online,united kingdom,https //www.findamasters.com/masters-degrees/c...


#[2.0.1]

In [54]:
raw_fees= pd.DataFrame(processed_df['fees'])

In [55]:
raw_fees.head()

,fees
0,please see university website information fees...
1,"uk £12,500 total international £28,750 total"
2,please see university website information fees...
3,please see university website information fees...
4,cost per 10 credits £722 10 alumni discount pa...


In [56]:
pattern = r' (?P<symbol>[^\d\s])(?P<value>\d{1,3}(?:,\d{3})*(?:\.\d{2})?)'

def return_cost(stri):
    # Search for the pattern in the string
    match = re.findall(pattern, stri)

    if not match:
        return None

    v = [value for currency, value in match]
   # print(v)
    max_v = max(v)
    merged_list = f'{match[0][0]}{max_v}'
    return merged_list


#raw_fees['fees'] = raw_fees['fees'].apply(lambda x: None if not bool(re.search(r'\d', x)) else x)
text_columns = processed_df.select_dtypes(include='object').columns
# Return the list of currency symbols and costs encountered in 'fees' field
raw_fees['fees'] = raw_fees['fees'].apply(lambda x: return_cost(x) if x is not None else None)




In [57]:
raw_fees.head()

,fees
0,None
1,"£28,750"
2,None
3,None
4,£722


In [58]:
# Your ExchangeRate-API key
api_key = '40f223580924eaf7a1eb4ee0'

# Fetch exchange rates from the API for all currencies against USD
api_url = f'https://open.er-api.com/v6/latest/USD?apikey={api_key}'
response = requests.get(api_url)
data = response.json()
exchange_rates = data['rates']

# Define a mapping between currency symbols in your data and API symbols
currency_symbol_mapping = {
    '£': 'GBP',
    '€': 'EUR',
    '$': 'USD',
    '¥': 'JPY'
    # Add more currency symbols as needed
}

# Function to convert any currency to the common currency (USD in this case)
def convert_to_common_currency(row, target_currency='USD'):
    if pd.isna(row['fees']):
        return None
    try:
        # Extract the currency symbol from the string
        currency_symbol = row['fees'][0]

        # Map the currency symbol to the API symbol
        api_currency_symbol = currency_symbol_mapping.get(currency_symbol)

        if not api_currency_symbol:
            return None

        # Extract the exchange rate from the pre-fetched rates
        exchange_rate = exchange_rates[api_currency_symbol]

        # Remove the currency symbol and commas, then convert to float
        amount = float(row['fees'][1:].replace(',', ''))

        # Convert to USD using the obtained exchange rate
        amount_target_currency = amount/(exchange_rate)
        return round(amount_target_currency,2)

    except Exception as e:
       return None


In [59]:
# Apply the conversion function to the 'fees' column
raw_fees['fees'] = raw_fees.apply(convert_to_common_currency, axis=1)

# Rename the column to indicate the common currency
common_currency_code = 'USD'
raw_fees.rename(columns={'fees': f'fees ({common_currency_code})'}, inplace=True)

In [60]:
raw_fees.head()

,fees (USD)
0,NaN
1,35255.96
2,NaN
3,NaN
4,885.38


susbituing and renaming the new column with the appropriate currance values

In [61]:
processed_df['fees']= raw_fees['fees (USD)']
processed_df.rename(columns={'fees': 'fees (USD)'}, inplace=True)


In [62]:
processed_df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees (USD),modality,duration,city,administration,country,url
0,3d design virtual environments msc,glasgow caledonian university,school engineering built environment,full time,3d visualis anim play role mani areas popular ...,september,NaN,msc,1 year full-time,glasgow,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...
1,air quality solutions msc,university leeds,institute transport studies,full time,7 million peopl estim die everi year due harm ...,september,35255.96,msc,1 year full time 2 3 years part-time,leeds,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...
2,analytical toxicology msc,king college london,faculty life sciences medicine,full time,analyt toxicolog msc uniqu studi cours integr ...,see course,NaN,msc,full-time one year,london,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...
3,applied computer science artificial intelligen...,university bradford,faculty engineering digital technologies,full time,comput scienc foundat mani excit cutting-edg t...,september january,NaN,msc,1 year full time 2 years part time,bradford,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...
4,applied economics banking financial markets on...,university bath,university bath online,part time,polit uncertainti financ recruit demands econo...,september january,885.38,msc,2 years 6 months full time,bath,online,united kingdom,https //www.findamasters.com/masters-degrees/c...


#[2.1.1]

In [137]:
vocabulary = set()

In [143]:
processed_df['description'] = processed_df.description.apply(lambda row: word_tokenize(row))

In [127]:
processed_df.description.head()

0    [3d, visualis, anim, play, role, mani, areas, ...
1    [7, million, peopl, estim, die, everi, year, d...
2    [analyt, toxicolog, msc, uniqu, studi, cours, ...
3    [comput, scienc, foundat, mani, excit, cutting...
4    [polit, uncertainti, financ, recruit, demands,...
Name: description, dtype: object

In [144]:
processed_df.description.apply(lambda row: [vocabulary.add(word) for word in row])

0       [None, None, None, None, None, None, None, Non...
1       [None, None, None, None, None, None, None, Non...
2       [None, None, None, None, None, None, None, Non...
3       [None, None, None, None, None, None, None, Non...
4       [None, None, None, None, None, None, None, Non...
                              ...                        
5974    [None, None, None, None, None, None, None, Non...
5975    [None, None, None, None, None, None, None, Non...
5976    [None, None, None, None, None, None, None, Non...
5977    [None, None, None, None, None, None, None, Non...
5978    [None, None, None, None, None, None, None, Non...
Name: description, Length: 5979, dtype: object

In [145]:
vocabulary

{'esa',
 'industrialis',
 'generalists',
 'easi',
 'spend',
 'second',
 'industrially-fund',
 'ifm',
 '12th',
 'undergo',
 'botnet',
 'beat',
 'amba-accredit',
 'seven-',
 'yet',
 'dialogue',
 'avi',
 'planet',
 'materialis',
 'well-earn',
 'fellow',
 'treatments',
 'pairings',
 'pavia',
 'ubiquit',
 'station',
 'interplay',
 'speech',
 'induct',
 'reintroduction',
 'idea',
 'africa',
 'animation',
 'beneath',
 'two-semest',
 'perspective',
 'strategically-design',
 'teamwork',
 'influences',
 'richardson',
 'workflows',
 'nelson',
 'manage-',
 'retir',
 'co-organ',
 'iggi',
 'design-bas',
 'controller',
 'indisput',
 'designer',
 'galvan',
 'immunotherapy',
 'einem',
 'psycho-pharmacolog',
 'ha',
 'har',
 'methodologies',
 'moore',
 'endang',
 'statistics',
 'cosmetics',
 'hbc',
 'stock-broking',
 'large–scal',
 'midwifery',
 'pspp',
 'cem',
 'mobility',
 'heading',
 'comets',
 '2026',
 'peoples',
 'torino',
 'vibrant',
 'qu',
 'airlin',
 'products',
 'fast',
 '3',
 'censu',
 'ableism

 Assign unique ID to each term in the vocabulary

In [149]:
terms = pd.DataFrame(data=list(vocabulary), columns=['term'])

In [150]:
terms.loc[0,'term']

'esa'

In [151]:
tqdm.pandas()

In [152]:
terms['reverse'] = terms.term.progress_apply(lambda item: list(processed_df.loc[processed_df.description.apply(lambda row: item in row)].index))

  0%|          | 0/12557 [00:00<?, ?it/s]

In [154]:
terms.loc[1000:1015]

,term,reverse
1000,tailor-mad,"[1595, 2018, 2857, 5005, 5973]"
1001,ispp,[3826]
1002,immun,"[73, 74, 184, 186, 326, 327, 2316, 2877, 3285,..."
1003,stubbornli,[3025]
1004,vertebr,[4300]
1005,carbon-neutr,[2749]
1006,universidad,"[1366, 2255, 2258, 4116]"
1007,pow,"[1830, 1854, 1860, 1960, 2959]"
1008,view/download,[1843]
1009,morpholog,"[3202, 4875]"


In [155]:
terms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12557 entries, 0 to 12556
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   term     12557 non-null  object
 1   reverse  12557 non-null  object
dtypes: object(2)
memory usage: 196.3+ KB


In [160]:
terms.head()

,term,reverse
0,esa,[2676]
1,industrialis,"[1417, 2150, 2841]"
2,generalists,"[3884, 5009]"
3,easi,"[1541, 1813, 4191]"
4,spend,"[104, 380, 407, 498, 627, 857, 1113, 1141, 139..."


dropping a column to have the reqested format of the function

In [167]:
term1 = terms.copy()
term1 = terms.drop(columns=['term'], axis=1)

In [168]:
term1.head()

,reverse
0,[2676]
1,"[1417, 2150, 2841]"
2,"[3884, 5009]"
3,"[1541, 1813, 4191]"
4,"[104, 380, 407, 498, 627, 857, 1113, 1141, 139..."


In [164]:
# Writing to a file
file_path = '/content/drive/MyDrive/HM3-ADM/HW3_ADM/vocabulary.json'
term1.to_json(file_path, orient='records', lines=True)